<a href="https://colab.research.google.com/github/sundarjhu/EscueladeVerano_2021/blob/main/Escuela_de_Verano_2021_Demo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Accessing data from SIMBAD, demonstrating ADQL basics**
---



---



In [ ]:
from astropy.table import Table, join
import numpy as np
!pip install pyvo
import pyvo as vo

## Create a table of source identifiers, save to VOTable. This will serve as input for our queries.

In [ ]:
id = ['lam Vel', 'IRAS 16205-2626', 'eta Car', 'alp Cen', 'IRC +10216'] #unique identifiers for three stars
hemisphere = ['Southern', 'Southern', 'Southern', 'Southern', 'Northern'] #which celestial hemisphere?
input_tab = Table([id, hemisphere], names = ('ID', 'hemisphere'))
input_tab.pprint()
input_tab.write('input_tab.vot', format = 'votable', overwrite = True)

### Automatically obtain the url for the SIMBAD TAP server by doing a regsearch using PyVO

In [ ]:
r = vo.regsearch(servicetype = 'tap', keywords = ['Simbad'])
SIMBAD_service = r[0].service

In [ ]:
# This is the url to the TAP server
SIMBAD_service.baseurl

In [ ]:
#Select all columns from the first row of basic data
query = """ SELECT top 1 * FROM basic """ # change "1" to any number to view that many rows in the output
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results.colnames
#You can use this to inspect the column names and their content in the table

In [ ]:
#How many rows are there in this table?
query = """ select count(coo_bibcode) as NUMROWS from basic"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results

In [ ]:
#HEY, waitaminit! I asked for the column to be in all caps, why isn't it?
#Because ADQL is case-insensitive unless you enclose the argument in quotes
query = """ select count(coo_bibcode) as "NUMROWS" from basic"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results

In [66]:
#SUBSETS with WHERE: How many rows are there in this table with DEC < 0?
query = """ select count(coo_bibcode) as NUMROWS from basic WHERE dec < 0"""
results = SIMBAD_service.search(query).to_table() #submit the ADQL query to the SIMBAD TAP server, and convert the response into an astropy table
results


#query only a selection of columns for one source
query = """
    select b.ids from basic a, ids b where a.main_id = 'lam Vel' and a.oid = b.oidref
"""
r_SIMBAD = SIMBAD_service.search(query).to_table()


# There are three important tables in SIMBAD: `basic`, `ident`, and `ids`. 

# `basic` contains the basic information for each source (positions, velocities, photometry, redshift, etc.). If a source is known by many names, there is an entry for EACH such name recognized by SIMBAD.

# Each source in these tables is identified by a unique identifier, a long integer called *oid* (in `basic`) and *oidref* (in `ident`, `ids`). You can connect a source to its various names using these two columns.

# You can obtain a list of references for the sources in the same way using the tables `ref` and `has_ref`.

In [ ]:
# Get some information from the basic table for the object with name 'IRC +10216', which is NOT its standard SIMBAD name.
#
query = """
-- Basic data from an object given one of its identifiers.
SELECT basic.OID,
       RA,
       DEC,
       main_id AS "Main identifier",
       coo_bibcode AS "Coord Reference",
       nbref AS "NbReferences",
       plx_value as "Parallax",
       rvz_radvel as "Radial velocity",
       galdim_majaxis,
       galdim_minaxis,
       galdim_angle AS "Galaxy ellipse angle"
FROM basic JOIN ident ON oidref = oid
WHERE id = 'IRC +10216'
"""
results = SIMBAD_service.search(query).to_table()
results

### Get all the identifiers for one source

In [ ]:
query = """select t1.id, t2.ids from ident as t1, ids as t2 where t1.id = 'IRC +10216' and t1.oidref = t2.oidref"""
results = SIMBAD_service.search(query).to_table()
results

# How to upload a table!
### Get all the identifiers for a list of sources 

In [ ]:
input_tab

In [ ]:
query = """select mytable.id, t2.ids from mytable, ident as t1, ids as t2 where t1.id = mytable.ID and t1.oidref = t2.oidref"""
results = SIMBAD_service.search(query, uploads = {'mytable': 'input_tab.vot'}).to_table()
results

In [ ]:
#Cone search
query = """
-- Display basic data about objects contained in a given circle and whose mag B < 9.0.
SELECT basic.OID,
       RA,
       DEC,
       main_id AS "Main identifier",
       coo_bibcode AS "BiblioReference",
       nbref AS "NbReferences",
       plx_value as "Parallax",
       rvz_radvel as "Radial velocity",
       galdim_majaxis,
       galdim_minaxis,
       galdim_angle AS "Galaxy ellipse angle"
FROM basic JOIN flux ON oidref = oid
WHERE filter = 'B'
      AND flux < 9.0
      AND CONTAINS(POINT('ICRS', RA, DEC), CIRCLE('ICRS', 10, 5, 1)) = 1
ORDER BY "Main identifier"
"""
results = SIMBAD_service.search(query).to_table()
results